**Feature pipeline:**

requirements:

In [ ]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Load dataset

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "tr", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "tr", split="test", use_auth_token=True)

print(common_voice)

Remove irrelevant features in audio data

In [ ]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

Prepare Feature Extractor, Tokenizer and Data

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Turkish", task="transcribe")

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Turkish", task="transcribe")

Downsample the data


In [ ]:
print(common_voice["train"][0])

In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
print(common_voice["train"][0])

In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)

In [ ]:
common_voice

Upload common_voice which is the processed data to google drive:

In [ ]:
!pip install pydrive

In [ ]:
from pydrive.drive import GoogleDrive
from pydrive.auth import GoogleAuth
import os

Save the dataset to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os.path
os.makedirs("/content/drive/MyDrive/common_voice/train")
os.makedirs("/content/drive/MyDrive/common_voice/test")

In [ ]:
common_voice["train"].save_to_disk("/content/drive/MyDrive/common_voice/train")
common_voice["test"].save_to_disk("/content/drive/MyDrive/common_voice/test")